In [1]:
from GNNModels.Models import *

import torch
from tqdm import tqdm
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

/opt/homebrew/anaconda3/envs/DIG_GXAI/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Importing the Model

In [2]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset_name = "Cora"

dataset = Planetoid(root='/tmp/Planetoid', name=dataset_name, transform=NormalizeFeatures())
data = dataset[0]  # Get the first graph object.

In [4]:
# # This is temporary model training, will be replaced with improting pretrained model, having problems with it currently

# from torch_geometric.nn import GCNConv

# class GCN(torch.nn.Module):
#     def __init__(self, hidden_channels):
#         super().__init__()
#         torch.manual_seed(1)
#         self.conv1 = GCNConv(dataset.num_features, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index)
#         x = x.relu()
#         #x = F.dropout(x, p=0.5, training=self.training)
#         x = self.conv2(x, edge_index)
#         return x

# model = GCN(hidden_channels=16)

# model = GCN(hidden_channels=32)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# criterion = torch.nn.CrossEntropyLoss()

# def train():
#       model.train()
#       optimizer.zero_grad()  # Clear gradients.
#       out = model(data.x, data.edge_index)  # Perform a single forward pass.
#       loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
#       val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
#       loss.backward()  # Derive gradients.
#       optimizer.step()  # Update parameters based on gradients.
#       return loss, val_loss

# def test():
#       model.eval()
#       out = model(data.x, data.edge_index)
#       pred = out.argmax(dim=1)  # Use the class with highest probability.
#       test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
#       test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
#       return test_acc, out


# for epoch in range(200):
#     loss, val_loss = train()
#     if epoch%200 == 0:
#           print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Validation loss: {val_loss:.4f}')

# test_acc, out = test()
# print(f'Test Accuracy: {test_acc:.4f}')

Epoch: 000, Loss: 1.9467, Validation loss: 1.9455
Test Accuracy: 0.8080


In [8]:
# Using pretrained model from GNNModels

model=get_model_pretrained('GCN','Cora')
criterion=torch.nn.CrossEntropyLoss()


# Explanation Methods

In [6]:
# %cd GrapthXAI-main
# !pip insall -e .

from graphxai.explainers import GNNExplainer, PGExplainer, IntegratedGradExplainer, PGMExplainer

# the ones below we want to use from different libraries
from graphxai.explainers import GNN_LRP, CAM

# need to also use subgraph x from DIG


## GNN Explainer and PGE Explainer

In [7]:
# GNN Explainer - discrete mask of node imp, soft mask of edge imp

gnnexp = GNNExplainer(model)

def gnn_imp_nodes(node_idx):

    node_exp = gnnexp.get_explanation_node(node_idx = node_idx, x = data.x, edge_index = data.edge_index)

    imp_nodes = []

    for k in node_exp.node_reference.keys():

        if node_exp.node_imp[node_exp.node_reference[k]].item() == 1:

            imp_nodes.append(k)

    return imp_nodes

# PGE Explainer - discrete maks of node imp, discrete mask of edge imp

# needs name of emb layer of the model
pgex = PGExplainer(model, emb_layer_name = 'conv2',  max_epochs = 500, lr = 0.01)
pgex.train_explanation_model(data)

def pge_imp_nodes(node_idx):

    node_exp = pgex.get_explanation_node(node_idx = node_idx, x = data.x, edge_index = data.edge_index)

    imp_nodes = []

    for k in node_exp.node_reference.keys():

        if node_exp.node_imp[node_exp.node_reference[k]].item() == 1:

            imp_nodes.append(k)

    return imp_nodes

140it [00:00, 756.22it/s]
140it [00:00, 747.14it/s]
140it [00:00, 945.36it/s]
140it [00:00, 904.54it/s]
140it [00:00, 626.60it/s]
140it [00:00, 811.79it/s]
140it [00:00, 878.98it/s]
140it [00:00, 881.60it/s]
140it [00:00, 937.04it/s]
140it [00:00, 922.92it/s]
140it [00:00, 758.60it/s]
140it [00:00, 765.39it/s]
140it [00:00, 832.97it/s]
140it [00:00, 805.57it/s]
140it [00:00, 703.56it/s]
140it [00:00, 788.47it/s]
140it [00:00, 934.79it/s]
140it [00:00, 872.82it/s]
140it [00:00, 969.45it/s]
140it [00:00, 950.71it/s]
140it [00:00, 1001.72it/s]
140it [00:00, 996.94it/s] 
140it [00:00, 989.06it/s]
140it [00:00, 1050.84it/s]
140it [00:00, 1051.26it/s]
140it [00:00, 1049.12it/s]
140it [00:00, 1040.04it/s]
140it [00:00, 1049.68it/s]
140it [00:00, 992.96it/s] 
140it [00:00, 750.02it/s]
140it [00:00, 899.60it/s]
140it [00:00, 889.25it/s]
140it [00:00, 954.75it/s] 
140it [00:00, 897.32it/s]

training time is 82.619s


## Integrated Gradients and PGM Explainer

In [13]:
# Integrated gradients - soft mask of edge imp

igex = IntegratedGradExplainer(model, criterion=criterion)

def ig_imp_nodes(node_idx):

    node_exp = igex.get_explanation_node(node_idx = node_idx, x = data.x, edge_index = data.edge_index, y = data.y)

    imp_nodes = []

    mask = torch.sigmoid(node_exp.node_imp) >= 0.5

    for k in node_exp.node_reference.keys():

        if mask[node_exp.node_reference[k]].item() == 1:
        
            imp_nodes.append(k)

    return imp_nodes

# PGME Explainer - discrete mask of node imp, randomised, can get ranking as well by asking for top 1 then 2 and so on

pgm = PGMExplainer(model, explain_graph=False)

def pgm_imp_nodes(node_idx, top = None):

    np.random.seed(1998)

    if top == None:

        node_exp = pgm.get_explanation_node(node_idx = node_idx, x = data.x, edge_index = data.edge_index)

    else: 

        node_exp = pgm.get_explanation_node(node_idx = node_idx, x = data.x, edge_index = data.edge_index, top_k_nodes=top)

    imp_nodes = []

    for k in node_exp.node_reference.keys():

        if node_exp.node_imp[node_exp.node_reference[k]].item() == 1:
        
            imp_nodes.append(k)

    return imp_nodes

## CAM

In [8]:
# CAM - soft mask of node importanct

camex = CAM(model)

def cam_imp_nodes(node_idx):

    node_exp = camex.get_explanation_node(node_idx = node_idx, x = data.x, edge_index = data.edge_index, y = data.y)

    imp_nodes = []

    mask = torch.sigmoid(node_exp.node_imp) >= 0.5

    for k in node_exp.node_reference.keys():

        if mask[node_exp.node_reference[k]].item() == 1:
        
            imp_nodes.append(k)

    return imp_nodes

